In [2]:
import os
import xarray as xr
import numpy as np
import json
import matplotlib.pyplot as plt
import pyqg
from collections import defaultdict
import glob
from plot_helpers import figure_grid
import itertools
import math
from scipy.stats import pearsonr

%matplotlib inline

In [3]:
test_dir = '/scratch/zanna/data/pyqg/64_256/test'
ds = xr.open_mfdataset(f"{test_dir}/*/lores.nc", combine="nested", concat_dim="run")

ds['relative_vorticity'] = (
    ds.v.differentiate('x') - ds.u.differentiate('y')
)
ds['divergence'] = (
    ds.u.differentiate('x') + ds.v.differentiate('y')
)
ds['shearing_deformation'] = (
    ds.u.differentiate('y') + ds.v.differentiate('x')
)
ds['stretching_deformation'] = (
    ds.u.differentiate('x') - ds.v.differentiate('y')
)

In [4]:
ds['u_forcing_physical_parameterization_eq5'] = (
    (ds.relative_vorticity**2).differentiate('x')
    - (ds.relative_vorticity * ds.shearing_deformation).differentiate('x')
    + (ds.relative_vorticity * ds.stretching_deformation).differentiate('y')   
)

ds['v_forcing_physical_parameterization_eq5'] = (
    (ds.relative_vorticity * ds.stretching_deformation).differentiate('x')
    + (ds.relative_vorticity**2).differentiate('y')
    + (ds.relative_vorticity * ds.shearing_deformation).differentiate('y')
)

ds['u_forcing_physical_parameterization_eq6'] = (
    (-ds.relative_vorticity*ds.shearing_deformation).differentiate('x')
    + (ds.relative_vorticity*ds.stretching_deformation).differentiate('y')
    + 0.5*(
        ds.relative_vorticity**2
        + ds.shearing_deformation**2
        + ds.stretching_deformation**2
    ).differentiate('x')  
)

ds['v_forcing_physical_parameterization_eq6'] = (
      (ds.relative_vorticity*ds.shearing_deformation).differentiate('y')
    + (ds.relative_vorticity*ds.stretching_deformation).differentiate('x')
    + 0.5*(
        ds.relative_vorticity**2
        + ds.shearing_deformation**2
        + ds.stretching_deformation**2
    ).differentiate('y')  
)

In [7]:
from scipy.stats import linregress

for eq in ['eq5', 'eq6']:
    for coord in ['u','v']:
        print(f"{coord} forcing physical parameterization correlation ({eq} of ZB2020)")
        for z in [0,1]:
            pred = np.array(ds[f"{coord}_forcing_physical_parameterization_{eq}"].isel(lev=z).data.reshape(-1))
            true = np.array(ds[f"{coord}_forcing_advection"].isel(lev=z).data.reshape(-1))
            res = linregress(pred, true)
            print(f"  {['upper','lower'][z]} layer: r={np.abs(res.rvalue):.2f} (slope={res.slope}, inter={res.intercept})")

u forcing physical parameterization correlation (eq5 of ZB2020)
  upper layer: r=0.21 (slope=-12087893.557416698, inter=1.0536144625369246e-13)
  lower layer: r=0.35 (slope=-18490218.88178126, inter=-1.30034398257813e-13)
v forcing physical parameterization correlation (eq5 of ZB2020)
  upper layer: r=0.20 (slope=-12126797.23322372, inter=-3.721772568638779e-12)
  lower layer: r=0.35 (slope=-18765223.769452106, inter=3.935995331232756e-14)
u forcing physical parameterization correlation (eq6 of ZB2020)
  upper layer: r=0.21 (slope=-19565025.147982173, inter=6.523999305914745e-13)
  lower layer: r=0.37 (slope=-32027163.944466308, inter=-1.0432412648684163e-13)
v forcing physical parameterization correlation (eq6 of ZB2020)
  upper layer: r=0.21 (slope=-19882697.466072824, inter=-4.207804007159038e-12)
  lower layer: r=0.39 (slope=-32689823.26071003, inter=1.7888873249724583e-14)
